In [26]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from datetime import datetime
from citipy import citipy

# Import the API key.
from config import weather_api_key

In [27]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=23ca3f74c011b5663163a441a6cbe2ed


In [28]:
# Create a new set of 2000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [29]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [30]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

742

In [31]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [32]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | hilo
Processing Record 3 of Set 1 | hermanus
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | dingle
Processing Record 6 of Set 1 | tuktoyaktuk
Processing Record 7 of Set 1 | cape town
Processing Record 8 of Set 1 | arman
Processing Record 9 of Set 1 | colesberg
Processing Record 10 of Set 1 | hobart
Processing Record 11 of Set 1 | mayo
Processing Record 12 of Set 1 | ribeira grande
Processing Record 13 of Set 1 | avarua
Processing Record 14 of Set 1 | samusu
City not found. Skipping...
Processing Record 15 of Set 1 | yellowknife
Processing Record 16 of Set 1 | touros
Processing Record 17 of Set 1 | cherskiy
Processing Record 18 of Set 1 | port alfred
Processing Record 19 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 20 of Set 1 | albany
Processing Record 21 of Set 1 | jamestown
Processing Record 22 of Set 1 | saint george
Processing Record 23 of Set 1 | camacha
Processing 

In [33]:
len(city_data)

682

In [37]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,Busselton,-33.65,115.33,60.01,68,100,13.33,AU,overcast clouds,2020-09-30 22:07:24
1,Hilo,19.73,-155.09,86.00,66,40,8.05,US,scattered clouds,2020-09-30 22:07:24
2,Hermanus,-34.42,19.23,51.01,86,96,8.01,ZA,light rain,2020-09-30 22:07:24
3,Ushuaia,-54.80,-68.30,32.00,93,90,20.80,AR,light snow,2020-09-30 22:07:25
4,Dingle,11.00,122.67,75.97,91,100,1.70,PH,overcast clouds,2020-09-30 22:07:25
5,Tuktoyaktuk,69.45,-133.04,41.00,86,20,8.05,CA,few clouds,2020-09-30 22:07:25
6,Cape Town,-33.93,18.42,55.00,81,75,27.51,ZA,light intensity shower rain,2020-09-30 22:03:13
7,Arman,59.70,150.17,43.66,75,77,5.21,RU,broken clouds,2020-09-30 22:07:26
8,Colesberg,-30.72,25.10,46.98,37,0,1.01,ZA,clear sky,2020-09-30 22:07:26
9,Hobart,-42.88,147.33,54.00,50,20,13.87,AU,few clouds,2020-09-30 22:05:39


In [39]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.65,115.33,60.01,68,100,13.33,overcast clouds
1,Hilo,US,19.73,-155.09,86.00,66,40,8.05,scattered clouds
2,Hermanus,ZA,-34.42,19.23,51.01,86,96,8.01,light rain
3,Ushuaia,AR,-54.80,-68.30,32.00,93,90,20.80,light snow
4,Dingle,PH,11.00,122.67,75.97,91,100,1.70,overcast clouds
5,Tuktoyaktuk,CA,69.45,-133.04,41.00,86,20,8.05,few clouds
6,Cape Town,ZA,-33.93,18.42,55.00,81,75,27.51,light intensity shower rain
7,Arman,RU,59.70,150.17,43.66,75,77,5.21,broken clouds
8,Colesberg,ZA,-30.72,25.10,46.98,37,0,1.01,clear sky
9,Hobart,AU,-42.88,147.33,54.00,50,20,13.87,few clouds


In [40]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")